In [1]:
import jupyter_black
from openai import OpenAI
import os
import json
from jinja2 import Template
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from dataclasses import dataclass
from IPython.display import Markdown


API_KEY = os.environ["DEEPSEEK_API_KEY"]
BASE_URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"

jupyter_black.load()

In [2]:
def llm_call(prompt: str, with_json_output: bool = False) -> tuple[str | dict, str]:
    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

    args = {
        "model": MODEL,
        "messages": [],
        "temperature": 0.0,
    }

    if with_json_output is True:
        json_prompt = """
        Output your response in JSON format with the keys specified in the prompt.
        Do not include any other text such as ```json or ```.
        The response should be directly parseable by json.loads.
        """.strip()
        args["messages"].append({"role": "system", "content": json_prompt})
        args["response_format"] = {"type": "json_object"}

    args["messages"].append({"role": "user", "content": prompt})
    response = client.chat.completions.create(**args)

    final_response = response.choices[0].message.content

    if with_json_output is True:
        return json.loads(final_response)

    return final_response

In [3]:
@dataclass
class Task:
    section_title: str
    rationale: str


@dataclass
class WorkerResult:
    section_title: str
    rationale: str
    result: str | None = None


class OrchestratorWorker:
    """Break down tasks and run them in parallel using worker LLMs."""

    def __init__(
        self,
        orchestrator_prompt: str,
        worker_prompt: str,
        synthesis_prompt: str,
    ):
        self.orchestrator_template = Template(orchestrator_prompt)
        self.worker_template = Template(worker_prompt)
        self.synthesis_template = Template(synthesis_prompt)

    def run_single_worker(self, main_task: str, task: Task) -> WorkerResult:
        """Run a single worker and return the result."""

        worker_input = self.worker_template.render(
            original_task=main_task,
            section_title=task.section_title,
            rationale=task.rationale,
        )

        print(f"\n[WORKER INPUT] Section: {task.section_title}\n{worker_input}\n")
        worker_response = llm_call(worker_input, with_json_output=True)
        result = worker_response["response"]

        print(f"\n[WORKER OUTPUT] Section: {task.section_title}\n{result}\n")

        return WorkerResult(
            section_title=task.section_title,
            rationale=task.rationale,
            result=result,
        )

    def process(self, task: str) -> list[WorkerResult]:
        """Run orchestrator and execute all workers in parallel."""

        input_str = self.orchestrator_template.render(task=task)
        print(f"\n[ORCHESTRATOR INPUT]\n{input_str}\n")

        orchestrator_output = llm_call(input_str, with_json_output=True)

        print(f"[ORCHESTRATOR OUTPUT]\nAnalysis:\n{orchestrator_output['analysis']}")
        print(f"\nTasks:\n{orchestrator_output['tasks']}\n")

        worker_tasks = [Task(**t) for t in orchestrator_output["tasks"]]
        print(f"[INFO] Total tasks: {len(worker_tasks)}\n")

        with ThreadPoolExecutor() as executor:
            results = list(
                tqdm(
                    executor.map(
                        lambda t: self.run_single_worker(task, t), worker_tasks
                    ),
                    total=len(worker_tasks),
                    desc="Running workers",
                )
            )

        return results

    def synthesize(self, task: str, worker_results: list[WorkerResult]) -> str:
        """Generate final report by synthesizing all worker results."""

        synthesis_input = self.synthesis_template.render(
            original_task=task,
            worker_results=worker_results,
        )

        print(f"\n[SYNTHESIS INPUT]\n{synthesis_input}\n")

        response = llm_call(synthesis_input, with_json_output=True)
        return response["response"]

In [4]:
ORCHESTRATOR_PROMPT = """
You're tasked with creating a report on a specific topic.

Break this down into 3-5 distinct sections that could each be written with a different focus.

Report topic: {{ task }}

Return your response in this JSON format:

{
  "analysis": "Summarize your understanding of the reporting goal and explain why breaking it into multiple sections with different focuses would be valuable. Highlight how each section emphasizes a different priority (e.g., technical depth, accessibility, narrative).",
  "tasks": [
    {
      "section_title": "Technical Overview",
      "rationale": "Explain why this section is important and what it will cover.",
    },
    {
      "section_title": "Practical Implications",
      "rationale": "Explain why this section is important and what it will cover.",
    }
  ]
}
"""

WORKER_PROMPT = """
You're generating a section of a report.

Topic: {{ original_task }}
Section title: {{ section_title }}
Rationale: {{ rationale }}


Write the content for this section using **Markdown formatting** (bold, italics, lists, code blocks, etc.) where appropriate, but **do not include a header/title** — just the body content.

Return your response in this JSON format:

{
  "response": "Your report content here, using Markdown formatting but without any section headers."
}
"""


SYNTHESIS_PROMPT = """
You're synthesizing multiple report sections into one cohesive final report.

Original topic: {{ original_task }}

These are the completed sections (in order):

{% for result in worker_results %}
Section title: {{ result.section_title }}
---
{{ result.result }}
{% endfor %}

Your task is to merge the sections into a well-flowing report. Use **Markdown formatting** where helpful, but **do not repeat the section titles**.

Ensure:
- The tone and style feel consistent throughout.
- Transitions between sections are smooth.
- Any overlap is minimized.
- The final output reads like a single unified piece.

Return your response in this JSON format:

{
  "response": "Your final merged report here, in Markdown."
}
"""

In [5]:
task = "Make an in-depth report that explains why biking has not been adopted in the City of Lisbon, and what can be done to encourage it."
orchestrator = OrchestratorWorker(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
    synthesis_prompt=SYNTHESIS_PROMPT,
)
results: list[WorkerResult] = orchestrator.process(task=task)
final_report: str = orchestrator.synthesize(task=task, worker_results=results)

print(final_report)


[ORCHESTRATOR INPUT]

You're tasked with creating a report on a specific topic.

Break this down into 3-5 distinct sections that could each be written with a different focus.

Report topic: Make an in-depth report that explains why biking has not been adopted in the City of Lisbon, and what can be done to encourage it.

Return your response in this JSON format:

{
  "analysis": "Summarize your understanding of the reporting goal and explain why breaking it into multiple sections with different focuses would be valuable. Highlight how each section emphasizes a different priority (e.g., technical depth, accessibility, narrative).",
  "tasks": [
    {
      "section_title": "Technical Overview",
      "rationale": "Explain why this section is important and what it will cover.",
    },
    {
      "section_title": "Practical Implications",
      "rationale": "Explain why this section is important and what it will cover.",
    }
  ]
}

[ORCHESTRATOR OUTPUT]
Analysis:
The reporting goal is 

Running workers:   0%|          | 0/5 [00:00<?, ?it/s]


[WORKER OUTPUT] Section: Policy and Governance
The City of Lisbon has faced several challenges in adopting biking as a mainstream mode of transportation, largely due to **policy and governance** issues. Below is an in-depth analysis of the current situation and potential solutions:

- **Existing Initiatives**: Lisbon has introduced some biking initiatives, such as the *Gira* bike-sharing system. However, these efforts are often **underfunded** and lack comprehensive integration with other transport modes.
- **Funding**: There is a noticeable **lack of dedicated funding** for biking infrastructure. Most transport budgets are allocated to roads and public transit, leaving biking projects under-resourced.
- **Regulatory Frameworks**: The city's regulatory environment does not sufficiently prioritize biking. For example, there are **no strict policies** mandating bike lanes in new urban developments.
- **Public Participation**: There is minimal **public engagement** in biking policy decis

Running workers:  20%|██        | 1/5 [00:21<01:27, 21.84s/it]


[WORKER OUTPUT] Section: Current Infrastructure and Challenges
The City of Lisbon faces several infrastructure challenges that hinder the widespread adoption of biking as a mode of transportation. Below is an analysis of the current situation and the obstacles that need to be addressed:

- **Bike Lanes**: The availability of bike lanes in Lisbon is limited and often fragmented, making it difficult for cyclists to navigate the city safely. *Existing bike lanes* are not uniformly distributed, with some areas well-served while others are completely lacking.

- **Parking Facilities**: Secure bike parking is scarce, discouraging potential cyclists who fear theft or vandalism. *The few available facilities* are often overcrowded or inconveniently located.

- **Topography**: Lisbon's hilly terrain presents a significant physical barrier to biking. *The steep inclines* in many parts of the city make cycling arduous for the average person, limiting its appeal as a convenient transport option.


Running workers: 100%|██████████| 5/5 [00:24<00:00,  4.91s/it]


[WORKER OUTPUT] Section: Public Perception and Cultural Attitudes
The adoption of biking in Lisbon faces significant challenges rooted in public perception and cultural attitudes. Despite the city's efforts to promote sustainable transport, several factors contribute to the reluctance among residents to embrace cycling as a primary mode of transportation.

- **Safety Concerns**: Many Lisbon residents perceive biking as unsafe, citing the city's hilly terrain, narrow streets, and heavy traffic. *"I don't feel safe riding a bike in Lisbon,"* is a common sentiment, especially among older populations and parents concerned about their children's safety.

- **Perceived Convenience**: Compared to other transport modes like cars, buses, and the metro, biking is often seen as less convenient. The city's public transport system is well-developed, offering a reliable alternative that doesn't require physical effort or exposure to the elements.

- **Cultural Preferences**: There's a strong car cu

The City of Lisbon faces several infrastructure challenges that hinder the widespread adoption of biking as a mode of transportation. The availability of bike lanes is limited and often fragmented, making it difficult for cyclists to navigate the city safely. Secure bike parking is scarce, discouraging potential cyclists who fear theft or vandalism. Additionally, Lisbon's hilly terrain presents a significant physical barrier to biking, with steep inclines in many parts of the city making cycling arduous for the average person. The coexistence of bikes with heavy traffic in areas without dedicated lanes poses safety risks, and there is minimal integration between biking and public transport systems.

Public perception and cultural attitudes further complicate the adoption of biking in Lisbon. Many residents perceive biking as unsafe due to the city's hilly terrain, narrow streets, and heavy traffic. Compared to other transport modes like cars, buses, and the metro, biking is often seen 

In [6]:
Markdown(final_report)

The City of Lisbon faces several infrastructure challenges that hinder the widespread adoption of biking as a mode of transportation. The availability of bike lanes is limited and often fragmented, making it difficult for cyclists to navigate the city safely. Secure bike parking is scarce, discouraging potential cyclists who fear theft or vandalism. Additionally, Lisbon's hilly terrain presents a significant physical barrier to biking, with steep inclines in many parts of the city making cycling arduous for the average person. The coexistence of bikes with heavy traffic in areas without dedicated lanes poses safety risks, and there is minimal integration between biking and public transport systems.

Public perception and cultural attitudes further complicate the adoption of biking in Lisbon. Many residents perceive biking as unsafe due to the city's hilly terrain, narrow streets, and heavy traffic. Compared to other transport modes like cars, buses, and the metro, biking is often seen as less convenient. There's also a strong car culture in Lisbon, with many viewing car ownership as a status symbol, while biking is sometimes associated with lower socioeconomic status or seen as a leisure activity rather than a practical means of transport.

Policy and governance issues also play a significant role in the slow adoption of biking. Existing initiatives, such as the *Gira* bike-sharing system, are often underfunded and lack comprehensive integration with other transport modes. There is a noticeable lack of dedicated funding for biking infrastructure, and the city's regulatory environment does not sufficiently prioritize biking. Minimal public engagement in biking policy decisions leads to a disconnect between community needs and governance actions.

A comparative analysis with other cities like Copenhagen and Amsterdam, which have extensive networks of bike lanes separated from motor traffic, highlights the potential for Lisbon to improve its biking infrastructure. Cities with high biking rates see cycling as a normal, everyday activity, thanks to public campaigns and education. Policy incentives, such as tax breaks for bike purchases and subsidized bike-sharing programs, along with seamless integration with public transport, have proven effective in other cities.

To encourage biking in Lisbon, a multi-faceted approach is essential. Infrastructure improvements should include expanding and connecting bike lanes, increasing secure bike parking facilities, and maintaining bike paths. Awareness campaigns can promote the benefits of biking, educate on safety, and encourage bike-to-work programs. Policy changes, such as subsidies for bike purchases, establishing car-free zones, and improving bike integration with public transport, are also crucial. By addressing these barriers, Lisbon can foster a more bike-friendly urban environment and move towards a more sustainable and healthy transport system.